<H1>Variable names refer to pitchers since this was cloned from the pitching script.<br><br>
Identifiers have been switching out for batters.</H1>

**The purpose of this notebook is to crawl through a list of batters and grab all seasons with a listed salary for the seasons between 1985-2017 (inclusive)**

In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
import os
import pickle
import string

from datetime import datetime

delay_speed = 3 # seconds

In [2]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


In [3]:
# Data cells
root_url = 'https://www.baseball-reference.com' #player url starts with a /

lower_bound_year = 1985
upper_bound_year = 2017

team_codes = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'FLA', 'HOU', 'KCR',
             'ANA', 'LAD', 'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF', 'STL', 'SEA', 'TBD',
             'TEX', 'TOR', 'WSN']



In [4]:
with open ('all_batting_urls', 'rb') as fp:
    all_pitches_urls = pickle.load(fp)

In [5]:
#Partition the players by alphabet so we can save in chunks in case something goes wrong (or we run out of memory)
all_pitches_urls = sorted(list(all_pitches_urls))
alphabet_strings = {url.split('/')[2] for url in all_pitches_urls}
partitioned_pitchers_by_alphabet = {alpha_char: [] for alpha_char in alphabet_strings}
for url in all_pitches_urls:
    partitioned_pitchers_by_alphabet[url.split('/')[2]].append(url)


In [6]:
def get_text_from_BR_url(url):
    driver.get(url)
    #if response.status_code == 200:
    return driver.page_source

def br_wait():
    time.sleep(delay_speed)
    
def get_deepest_node(soup_element):
    descendents = soup_element.descendents
    if descendents:
        return descendents[-1]
    else:
        return soup_element
    
def get_player_page_soup(player_url):
    full_url = root_url + player_url
    page_text = get_text_from_BR_url(full_url)
    if not page_text:
        return None
    
    soup = BeautifulSoup(page_text, "lxml")
    return soup

def is_valid_stat_row(row_soup):
    row_classes = set(row_soup.get("class"))
    # hidden rows are minor league rows, spacer rows are for years out of the league (injury, suspension, etc)
    invalid_row_classes = {'hidden','spacer'}
    # If these sets have an intersection, that means row_classes is not a stat row
    return len(invalid_row_classes.intersection(row_classes)) == 0

def id_from_url(url):
    #'/players/m/maddugr01.shtml'
    return url.split('/')[-1].split('.')[0]

In [7]:
#header scraping code
id_player_header = 'info'

def get_player_header(soup):
    return soup.find('div', id=id_player_header)

def get_name(soup):
    return soup.text
    
def get_throwing_hand(soup):
    throws_elem = soup.find('strong', text='Bats: ')
    return throws_elem.next_sibling.split()[0]

def get_weight_height(soup):
    height_weight = soup.find_all('span')
    #0 = height, 1 = weight
    return height_weight[0].text, height_weight[1].text

def get_weight_height_index(children):
    regular_index = 3
    index_if_misc_note = 4
    
    potential_height_weight = children[regular_index].find_all('span')
    if len(potential_height_weight) == 0:
        return index_if_misc_note
    else:
        return regular_index

def scrape_header(soup):
    header_soup = get_player_header(soup)
    
    anchor_element = None
    head_children = header_soup.findChildren(recursive=False)[0].findChildren(recursive=False)
    # Need to offset our indices for players without pictures
    first_child_classes = head_children[0].get("class")
    if first_child_classes is not None and 'media-item' in first_child_classes:
        anchor_element = head_children[1]
    else:
        anchor_element = head_children[0]
    
    children = anchor_element.findChildren(recursive=False)
    
    name_index = 0
    bats_throws_index = 2
    hw_index = get_weight_height_index(children)
    
    name_node = children[name_index]
    name = get_name(name_node)
    
    bats_throws_node = children[bats_throws_index]
    throwing_hand = get_throwing_hand(bats_throws_node)
    
    weight_height_node = children[hw_index]
    height, weight = get_weight_height(weight_height_node)
    return [name, throwing_hand, height, weight]


In [8]:
# Collecting Pitchers

id_pitching_table = 'batting_standard'
id_pitching_value = 'batting_value'

def get_pitching_standard_table(soup):
    return soup.find('table', id=id_pitching_table)

def get_pitching_value_table(soup):
    return soup.find('table', id=id_pitching_value)

def scrape_salaries(soup):
    year_salary_dict = {}
    value_table_soup = get_pitching_value_table(soup)
    
    if value_table_soup:
        for row in value_table_soup.find('tbody').findChildren(recursive=False): # tr elements
            if is_valid_stat_row(row):
                cells = row.findChildren(recursive=False)
                year = int(cells[0].text)
                # First check if this is a row for 1985-2017
                if year >= lower_bound_year and year <= upper_bound_year:
                    # last cell is awards, second to last is position, 3rd to last is salary
                    salary = cells[-3].text
                    if salary:
                        stripped_slary = salary.strip()
                        if len(stripped_slary) > 0:
                            year_salary_dict[year] = stripped_slary
                    
    return year_salary_dict

def scrape_yearly_pitching_stats(soup):
    found_salaries = scrape_salaries(soup)
    # we only want to bother with the stat table if they have at least one season with a listed salary
    if found_salaries:
        standard_pitching_stats = scrape_standard_pitching_stats(soup, found_salaries)
        return standard_pitching_stats
    else:
        return None

def scrape_standard_pitching_stats(soup, salaries):
    table = get_pitching_standard_table(soup)
    if table:
        rows_of_stats = []
        for row in table.find('tbody').findChildren(recursive=False): # tr elements
            if is_valid_stat_row(row):
                cells = row.findChildren(recursive=False)
                year = int(cells[0].text)
                # don't bother scraping the row unless we have a salary for that year
                if year in salaries:
                    stats_for_year = [get_deepest_node(stat).text for stat in cells[1:]]
                    #add the year as the first element
                    stats_for_year.insert(0, year)
                    #add the salary as the last element
                    stats_for_year.append(salaries[year])
                    
                    rows_of_stats.append(stats_for_year)
                    
        return rows_of_stats
    return None
                    
    
def scrape_pitcher_career_stats(player_url):
    soup = get_player_page_soup(player_url)
    yearly_stats = scrape_yearly_pitching_stats(soup)
    # yearly_stats will be None if no salary data exists (cases such as players who only ever got a midseason callup)
    if yearly_stats:
        player_id = id_from_url(player_url)
        header = scrape_header(soup)
        header.insert(0,player_id)
        # now we merge the pitcher_header level stats (height, weight, etc) with the yearly stats for pandas
        full_list = [header + stat_list for stat_list in yearly_stats]
        return full_list
    else:
        return None

In [9]:
def scrape_pitcher_standard_headers():
    soup = get_player_page_soup('/players/p/pujolal01.shtml')
    table = get_pitching_standard_table(soup)
    if table:
        children = table.find('thead').find('tr').findChildren()
        return [child.text for child in children]
    else:
        return None

In [10]:
driver = webdriver.Chrome(chromedriver)
stat_column_headers = scrape_pitcher_standard_headers()

player_main_information_headers = ['ID', 'NAME','BATS','HEIGHT','WEIGHT']
full_list_of_columns = player_main_information_headers + stat_column_headers + ['SALARY']
#make them all caps
full_list_of_columns = [x.upper() for x in full_list_of_columns]


time_benchmarking = ['START: {0}'.format(str(datetime.now()))]
for letter, urls in partitioned_pitchers_by_alphabet.items():
    
    full_stats = []
    for url in urls:
        print(url)
        new_stats = scrape_pitcher_career_stats(url)
        if new_stats is not None and new_stats:
            full_stats = full_stats + new_stats
        br_wait()
    
    pitchers_df = pd.DataFrame(full_stats, columns=full_list_of_columns)
    pitchers_df = pitchers_df.apply(pd.to_numeric, errors='ignore')
    csv_name = 'offense/batters_{0}.csv'.format(letter)
    pitchers_df.to_csv(csv_name)
    time_benchmarking.append('{0} COMPLETE: {1}'.format(letter, str(datetime.now())))

time_benchmarking.append('SCRAPE COMPLETE: {0}'.format(str(datetime.now())))


/players/l/labanjo01.shtml
/players/l/lacyle01.shtml
/players/l/ladenty01.shtml
/players/l/laforpe01.shtml
/players/l/lagami01.shtml
/players/l/lagarju01.shtml
/players/l/lahaibr01.shtml
/players/l/lairdbr01.shtml
/players/l/lairdge01.shtml
/players/l/lakeju01.shtml
/players/l/lakerti01.shtml
/players/l/lakest01.shtml
/players/l/lallibl01.shtml
/players/l/lamarry01.shtml
/players/l/lambda01.shtml
/players/l/lambja01.shtml
/players/l/lambmi01.shtml
/players/l/lamboan01.shtml
/players/l/lampkto01.shtml
/players/l/lanceri01.shtml
/players/l/landrce01.shtml
/players/l/landrke01.shtml
/players/l/landrti01.shtml
/players/l/laneja01.shtml
/players/l/langery01.shtml
/players/l/lankfra01.shtml
/players/l/lansfca01.shtml
/players/l/lansimi01.shtml
/players/l/laporma01.shtml
/players/l/larisje01.shtml
/players/l/larkiba01.shtml
/players/l/larkige01.shtml
/players/l/larkist02.shtml
/players/l/larocad01.shtml
/players/l/larocan01.shtml
/players/l/larocgr01.shtml
/players/l/larsobr01.shtml
/players/

/players/m/mcdante01.shtml
/players/m/mcdavra01.shtml
/players/m/mcdonda02.shtml
/players/m/mcdondo01.shtml
/players/m/mcdonja02.shtml
/players/m/mcdonjo03.shtml
/players/m/mcdonke01.shtml
/players/m/mcdouma01.shtml
/players/m/mcdowod01.shtml
/players/m/mcewijo01.shtml
/players/m/mcgeewi01.shtml
/players/m/mcgehca01.shtml
/players/m/mcginru01.shtml
/players/m/mcgrifr01.shtml
/players/m/mcgrite01.shtml
/players/m/mcguibi01.shtml
/players/m/mcguich01.shtml
/players/m/mcguiry01.shtml
/players/m/mcgwima01.shtml
/players/m/mcintti01.shtml
/players/m/mckayco01.shtml
/players/m/mckeewa01.shtml
/players/m/mckenmi01.shtml
/players/m/mcknije01.shtml
/players/m/mclemma01.shtml
/players/m/mclouna01.shtml
/players/m/mcmahry01.shtml
/players/m/mcmicgr01.shtml
/players/m/mcmilbi01.shtml
/players/m/mcnamji01.shtml
/players/m/mcneeje01.shtml
/players/m/mcpheda01.shtml
/players/m/mcraebr01.shtml
/players/m/mcraeha01.shtml
/players/m/mcreyke01.shtml
/players/m/meachbo01.shtml
/players/m/meadolo01.shtml
/

/players/c/cervemi01.shtml
/players/c/cespeyo01.shtml
/players/c/ceyro01.shtml
/players/c/chambad01.shtml
/players/c/chambal01.shtml
/players/c/chambch01.shtml
/players/c/chambji01.shtml
/players/c/chambwe01.shtml
/players/c/chapmke01.shtml
/players/c/chapmma01.shtml
/players/c/chapmtr01.shtml
/players/c/charlfr01.shtml
/players/c/chavean02.shtml
/players/c/chaveen01.shtml
/players/c/chaveer01.shtml
/players/c/chavera01.shtml
/players/c/chench01.shtml
/players/c/chiriro01.shtml
/players/c/chiselo01.shtml
/players/c/choicmi01.shtml
/players/c/choihe01.shtml
/players/c/choiji01.shtml
/players/c/choosh01.shtml
/players/c/chrisjo02.shtml
/players/c/chrisju01.shtml
/players/c/chrismc01.shtml
/players/c/chrisry01.shtml
/players/c/chrisst01.shtml
/players/c/churcry01.shtml
/players/c/cianfar01.shtml
/players/c/cintral01.shtml
/players/c/ciriape01.shtml
/players/c/cirilje01.shtml
/players/c/clapich01.shtml
/players/c/clappst01.shtml
/players/c/clarkbo04.shtml
/players/c/clarkbr02.shtml
/player

/players/r/rodriiv01.shtml
/players/r/rodrijo03.shtml
/players/r/rodrijo05.shtml
/players/r/rodrili01.shtml
/players/r/rodrilu01.shtml
/players/r/rodriru01.shtml
/players/r/rodrise01.shtml
/players/r/rodrist01.shtml
/players/r/rodrito01.shtml
/players/r/rodrivi01.shtml
/players/r/rodriyo01.shtml
/players/r/roeniga01.shtml
/players/r/roeniro01.shtml
/players/r/rogered01.shtml
/players/r/rogerja02.shtml
/players/r/rohdeda01.shtml
/players/r/rohliry01.shtml
/players/r/rohnda01.shtml
/players/r/rohrmda01.shtml
/players/r/rojasmi02.shtml
/players/r/rolensc01.shtml
/players/r/rolisna01.shtml
/players/r/rolliji01.shtml
/players/r/rollsda01.shtml
/players/r/romakja01.shtml
/players/r/romanja01.shtml
/players/r/romanto01.shtml
/players/r/romeral01.shtml
/players/r/romered01.shtml
/players/r/romerma01.shtml
/players/r/romerni01.shtml
/players/r/romerst01.shtml
/players/r/rominan01.shtml
/players/r/rominau01.shtml
/players/r/rominke01.shtml
/players/r/ronanma01.shtml
/players/r/rondojo02.shtml
/p

/players/t/thomacl02.shtml
/players/t/thomade01.shtml
/players/t/thomafr04.shtml
/players/t/thomago01.shtml
/players/t/thomeji01.shtml
/players/t/thompan02.shtml
/players/t/thompja01.shtml
/players/t/thompja02.shtml
/players/t/thompke01.shtml
/players/t/thompmi02.shtml
/players/t/thompri02.shtml
/players/t/thompro01.shtml
/players/t/thompry01.shtml
/players/t/thompsc01.shtml
/players/t/thomptr01.shtml
/players/t/thondi01.shtml
/players/t/thormsc01.shtml
/players/t/thornan01.shtml
/players/t/thornlo01.shtml
/players/t/thurmga01.shtml
/players/t/thursjo01.shtml
/players/t/tiffete01.shtml
/players/t/tillmru01.shtml
/players/t/tilsoch01.shtml
/players/t/timmooz01.shtml
/players/t/timpncl01.shtml
/players/t/tinglro01.shtml
/players/t/tinslle01.shtml
/players/t/tocajo01.shtml
/players/t/tolbema01.shtml
/players/t/tolenjo01.shtml
/players/t/tolesan01.shtml
/players/t/tollest01.shtml
/players/t/tollewa01.shtml
/players/t/tolmati01.shtml
/players/t/tomasya01.shtml
/players/t/tombean01.shtml
/pl

...

/players/b/bautida01.shtml
/players/b/bautijo02.shtml
/players/b/bautira01.shtml
/players/b/baxtemi01.shtml
/players/b/bayja01.shtml
/players/b/baylodo01.shtml
/players/b/beamotr01.shtml
/players/b/beanbi01.shtml
/players/b/beanebi01.shtml
/players/b/beckeri01.shtml
/players/b/beckhgo01.shtml
/players/b/beckhti01.shtml
/players/b/belchke01.shtml
/players/b/belkti01.shtml
/players/b/bellbu01.shtml
/players/b/bellda01.shtml
/players/b/bellde01.shtml
/players/b/belleal01.shtml
/players/b/bellge02.shtml
/players/b/bellhma01.shtml
/players/b/bellicl01.shtml
/players/b/bellico01.shtml
/players/b/bellira01.shtml
/players/b/belliro01.shtml
/players/b/bellja01.shtml
/players/b/belljo01.shtml
/players/b/belljo02.shtml
/players/b/bellju01.shtml
/players/b/bellmi01.shtml
/players/b/bellmi02.shtml
/players/b/belloed01.shtml
/players/b/bellte01.shtml
/players/b/belnovi01.shtml
/players/b/beltbr01.shtml
/players/b/beltrad01.shtml
/players/b/beltrca01.shtml
/players/b/beltren01.shtml
/players/b/beltre

/players/e/estalbo02.shtml
/players/e/estrajo01.shtml
/players/e/ethiean01.shtml
/players/e/eusebto01.shtml
/players/e/evansda01.shtml
/players/e/evansdw01.shtml
/players/e/evansni01.shtml
/players/e/evansph01.shtml
/players/e/evanste01.shtml
/players/e/evansto01.shtml
/players/e/everead01.shtml
/players/e/evereca01.shtml
/players/e/everito01.shtml
/players/e/exposlu01.shtml
/players/d/dahlda01.shtml
/players/d/dalenpe01.shtml
/players/d/dalesma01.shtml
/players/d/dallibr01.shtml
/players/d/damonjo01.shtml
/players/d/danieka01.shtml
/players/d/danksjo02.shtml
/players/d/dantoja01.shtml
/players/d/darnach01.shtml
/players/d/darnatr01.shtml
/players/d/darneja01.shtml
/players/d/darrmi02.shtml
/players/d/dascedo01.shtml
/players/d/datzje01.shtml
/players/d/daubabr01.shtml
/players/d/dauerri01.shtml
/players/d/daughja01.shtml
/players/d/daultda01.shtml
/players/d/davanje02.shtml
/players/d/davidan01.shtml
/players/d/davidcl02.shtml
/players/d/davidma01.shtml
/players/d/davidma02.shtml
/pla

/players/s/seabosc01.shtml
/players/s/seageco01.shtml
/players/s/seageky01.shtml
/players/s/searsto01.shtml
/players/s/seela01.shtml
/players/s/sefcike01.shtml
/players/s/segedro01.shtml
/players/s/seguida01.shtml
/players/s/seguife01.shtml
/players/s/segurje01.shtml
/players/s/seitzke01.shtml
/players/s/selbybi01.shtml
/players/s/selfto01.shtml
/players/s/selleju01.shtml
/players/s/selskst01.shtml
/players/s/semiema01.shtml
/players/s/servasc01.shtml
/players/s/severpe01.shtml
/players/s/sexsori01.shtml
/players/s/sextoch01.shtml
/players/s/shabaad01.shtml
/players/s/shaffri01.shtml
/players/s/sharpmi01.shtml
/players/s/shavejo01.shtml
/players/s/shawtr01.shtml
/players/s/sheafda01.shtml
/players/s/shealry01.shtml
/players/s/sheetan01.shtml
/players/s/sheetla01.shtml
/players/s/sheffga01.shtml
/players/s/shelbjo01.shtml
/players/s/sheldsc01.shtml
/players/s/sheltbe01.shtml
/players/s/sheltch01.shtml
/players/s/shephro01.shtml
/players/s/sheripa01.shtml
/players/s/shermda02.shtml
/play

/players/a/ariasge01.shtml
/players/a/ariasjo01.shtml
/players/a/armasma01.shtml
/players/a/armasto01.shtml
/players/a/arndtla01.shtml
/players/a/arroych01.shtml
/players/a/arrueba01.shtml
/players/a/asadora01.shtml
/players/a/ascheco01.shtml
/players/a/ashbyal01.shtml
/players/a/ashlebi01.shtml
/players/a/ashlene01.shtml
/players/a/asuajca01.shtml
/players/a/atkinga01.shtml
/players/a/aubremi01.shtml
/players/a/auderi01.shtml
/players/a/aurilri01.shtml
/players/a/ausmubr01.shtml
/players/a/austity01.shtml
/players/a/avenbr01.shtml
/players/a/averyxa01.shtml
/players/a/avilaal01.shtml
/players/a/avilemi01.shtml
/players/a/ayalabe01.shtml
/players/a/aybarer01.shtml
/players/a/aybarwi01.shtml
/players/a/ayraujo01.shtml
/players/a/azocaos01.shtml
/players/p/pachejo01.shtml
/players/p/pacioji01.shtml
/players/p/pacioto01.shtml
/players/p/padiljo01.shtml
/players/p/paganan01.shtml
/players/p/paglimi01.shtml
/players/p/pagnoma01.shtml
/players/p/pagnoto01.shtml
/players/p/palacre01.shtml
/pl

/players/h/hebneri01.shtml
/players/h/hechaad01.shtml
/players/h/hedgeau01.shtml
/players/h/heepda01.shtml
/players/h/heffebe01.shtml
/players/h/hegmabo01.shtml
/players/h/heintch01.shtml
/players/h/heisech01.shtml
/players/h/helfaer01.shtml
/players/h/helmswe01.shtml
/players/h/heltoto01.shtml
/players/h/hemonsc01.shtml
/players/h/hemphbr01.shtml
/players/h/hendeda01.shtml
/players/h/henderi01.shtml
/players/h/hendest01.shtml
/players/h/hendrge01.shtml
/players/h/hengeda01.shtml
/players/h/henlebo01.shtml
/players/h/hensodr01.shtml
/players/h/heredgu01.shtml
/players/h/hermach01.shtml
/players/h/hermije01.shtml
/players/h/hernaal01.shtml
/players/h/hernaan01.shtml
/players/h/hernaca01.shtml
/players/h/hernaca02.shtml
/players/h/hernace01.shtml
/players/h/hernace02.shtml
/players/h/hernadi01.shtml
/players/h/hernaen02.shtml
/players/h/hernago01.shtml
/players/h/hernajo01.shtml
/players/h/hernake01.shtml
/players/h/hernale01.shtml
/players/h/hernalu01.shtml
/players/h/hernama02.shtml
/p

In [11]:
time_benchmarking

['START: 2018-01-28 08:14:12.627219',
 'l COMPLETE: 2018-01-28 08:28:25.837250',
 'm COMPLETE: 2018-01-28 08:55:00.300824',
 'c COMPLETE: 2018-01-28 09:16:48.670430',
 'r COMPLETE: 2018-01-28 09:34:36.446667',
 'o COMPLETE: 2018-01-28 09:40:02.454201',
 'y COMPLETE: 2018-01-28 09:41:43.825544',
 'q COMPLETE: 2018-01-28 09:42:45.895113',
 'v COMPLETE: 2018-01-28 09:47:51.138399',
 'u COMPLETE: 2018-01-28 09:48:48.845961',
 't COMPLETE: 2018-01-28 09:59:02.156484',
 'w COMPLETE: 2018-01-28 10:10:55.994317',
 'j COMPLETE: 2018-01-28 10:19:46.581003',
 'g COMPLETE: 2018-01-28 10:36:53.219038',
 'n COMPLETE: 2018-01-28 10:42:33.961151',
 'f COMPLETE: 2018-01-28 10:51:13.301754',
 'k COMPLETE: 2018-01-28 10:58:20.709988',
 'i COMPLETE: 2018-01-28 10:59:54.763364',
 'b COMPLETE: 2018-01-28 11:26:08.561071',
 'e COMPLETE: 2018-01-28 11:30:30.273570',
 'd COMPLETE: 2018-01-28 11:44:29.512519',
 'z COMPLETE: 2018-01-28 11:45:44.622740',
 's COMPLETE: 2018-01-28 12:09:12.944696',
 'a COMPLETE: 20